## Titulo: N5 Now - Challenge Técnico

<br>

#### Autor: Nicolas San Martin

<br> 

#### Descripción: el desarrollo aborda el siguiente punto del challenge: 'Cargar los datasets utilizando Pandas y mantenerlos en formato parquet'.
<br>

#### Consideraciones iniciales previo al comienzo del desarrollo:
<br>

- Para cada uno de los esquemas definidos en los Dataframe's, no se hace validación de los datos, como por ejemplo, que una columna no tenga valores nulos (a través del párametro nullable), así como tampoco la adición de metadata.

- No se realiza manejo de excepciones.

- No se realiza limpieza de los nombres de las columnas.

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
path_datalake = "input/datalake/kaggle/covid"

file_name_country_wise_latest = "country_wise_latest"
file_name_covid_19_clean_complete_df = "covid_19_clean_complete"
file_name_day_wise_df = "day_wise"
file_name_full_grouped_df = "full_grouped"
file_name_usa_county_wise_df = "usa_county_wise"
file_name_worldometer_data_df = "worldometer_data"

#### Lectura de archivos csv

###### Siguiendo los lineamientos definidos en la descripción del challenge, en esta etapa no se tratan los tipos de datos de cada uno de los archivos, sino que se realiza el casteo de los datos posteriormente.- No se usa dtypes

In [3]:
country_wise_latest_df = pd.read_csv(f"{path_datalake}/{file_name_country_wise_latest}.csv")

covid_19_clean_complete_df = pd.read_csv(f"{path_datalake}/{file_name_covid_19_clean_complete_df}.csv")

day_wise_df = pd.read_csv(f"{path_datalake}/{file_name_day_wise_df}.csv")

full_grouped_df = pd.read_csv(f"{path_datalake}/{file_name_full_grouped_df}.csv")

usa_county_wise_df = pd.read_csv(f"{path_datalake}/{file_name_usa_county_wise_df}.csv")

worldometer_data_df = pd.read_csv(f"{path_datalake}/{file_name_worldometer_data_df}.csv")

#### Validación de lectura

In [4]:
country_wise_latest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country/Region          187 non-null    object 
 1   Confirmed               187 non-null    int64  
 2   Deaths                  187 non-null    int64  
 3   Recovered               187 non-null    int64  
 4   Active                  187 non-null    int64  
 5   New cases               187 non-null    int64  
 6   New deaths              187 non-null    int64  
 7   New recovered           187 non-null    int64  
 8   Deaths / 100 Cases      187 non-null    float64
 9   Recovered / 100 Cases   187 non-null    float64
 10  Deaths / 100 Recovered  187 non-null    float64
 11  Confirmed last week     187 non-null    int64  
 12  1 week change           187 non-null    int64  
 13  1 week % increase       187 non-null    float64
 14  WHO Region              187 non-null    ob

In [5]:
country_wise_latest_df.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


In [6]:
covid_19_clean_complete_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49068 entries, 0 to 49067
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  14664 non-null  object 
 1   Country/Region  49068 non-null  object 
 2   Lat             49068 non-null  float64
 3   Long            49068 non-null  float64
 4   Date            49068 non-null  object 
 5   Confirmed       49068 non-null  int64  
 6   Deaths          49068 non-null  int64  
 7   Recovered       49068 non-null  int64  
 8   Active          49068 non-null  int64  
 9   WHO Region      49068 non-null  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 3.7+ MB


In [7]:
covid_19_clean_complete_df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


In [8]:
day_wise_df.head()

,Date,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries
0,2020-01-22,555,17,28,510,0,0,0,3.06,5.05,60.71,6
1,2020-01-23,654,18,30,606,99,1,2,2.75,4.59,60.00,8
2,2020-01-24,941,26,36,879,287,8,6,2.76,3.83,72.22,9
3,2020-01-25,1434,42,39,1353,493,16,3,2.93,2.72,107.69,11
4,2020-01-26,2118,56,52,2010,684,14,13,2.64,2.46,107.69,13


In [9]:
full_grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35156 entries, 0 to 35155
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            35156 non-null  object
 1   Country/Region  35156 non-null  object
 2   Confirmed       35156 non-null  int64 
 3   Deaths          35156 non-null  int64 
 4   Recovered       35156 non-null  int64 
 5   Active          35156 non-null  int64 
 6   New cases       35156 non-null  int64 
 7   New deaths      35156 non-null  int64 
 8   New recovered   35156 non-null  int64 
 9   WHO Region      35156 non-null  object
dtypes: int64(7), object(3)
memory usage: 2.7+ MB


In [10]:
full_grouped_df.head()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,WHO Region
0,2020-01-22,Afghanistan,0,0,0,0,0,0,0,Eastern Mediterranean
1,2020-01-22,Albania,0,0,0,0,0,0,0,Europe
2,2020-01-22,Algeria,0,0,0,0,0,0,0,Africa
3,2020-01-22,Andorra,0,0,0,0,0,0,0,Europe
4,2020-01-22,Angola,0,0,0,0,0,0,0,Africa


In [11]:
usa_county_wise_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627920 entries, 0 to 627919
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   UID             627920 non-null  int64  
 1   iso2            627920 non-null  object 
 2   iso3            627920 non-null  object 
 3   code3           627920 non-null  int64  
 4   FIPS            626040 non-null  float64
 5   Admin2          626792 non-null  object 
 6   Province_State  627920 non-null  object 
 7   Country_Region  627920 non-null  object 
 8   Lat             627920 non-null  float64
 9   Long_           627920 non-null  float64
 10  Combined_Key    627920 non-null  object 
 11  Date            627920 non-null  object 
 12  Confirmed       627920 non-null  int64  
 13  Deaths          627920 non-null  int64  
dtypes: float64(3), int64(4), object(7)
memory usage: 67.1+ MB


In [12]:
usa_county_wise_df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Deaths
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",1/22/20,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",1/22/20,0,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",1/22/20,0,0
3,63072001,PR,PRI,630,72001.0,Adjuntas,Puerto Rico,US,18.180117,-66.754367,"Adjuntas, Puerto Rico, US",1/22/20,0,0
4,63072003,PR,PRI,630,72003.0,Aguada,Puerto Rico,US,18.360255,-67.175131,"Aguada, Puerto Rico, US",1/22/20,0,0


In [13]:
worldometer_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country/Region    209 non-null    object 
 1   Continent         208 non-null    object 
 2   Population        208 non-null    float64
 3   TotalCases        209 non-null    int64  
 4   NewCases          4 non-null      float64
 5   TotalDeaths       188 non-null    float64
 6   NewDeaths         3 non-null      float64
 7   TotalRecovered    205 non-null    float64
 8   NewRecovered      3 non-null      float64
 9   ActiveCases       205 non-null    float64
 10  Serious,Critical  122 non-null    float64
 11  Tot Cases/1M pop  208 non-null    float64
 12  Deaths/1M pop     187 non-null    float64
 13  TotalTests        191 non-null    float64
 14  Tests/1M pop      191 non-null    float64
 15  WHO Region        184 non-null    object 
dtypes: float64(12), int64(1), object(3)
memory u

In [14]:
worldometer_data_df.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,3.311981e+08,5032179,NaN,162804.0,NaN,2576668.0,NaN,2292707.0,18296.0,15194.0,492.0,63139605.0,190640.0,Americas
1,Brazil,South America,2.127107e+08,2917562,NaN,98644.0,NaN,2047660.0,NaN,771258.0,8318.0,13716.0,464.0,13206188.0,62085.0,Americas
2,India,Asia,1.381345e+09,2025409,NaN,41638.0,NaN,1377384.0,NaN,606387.0,8944.0,1466.0,30.0,22149351.0,16035.0,South-EastAsia
3,Russia,Europe,1.459409e+08,871894,NaN,14606.0,NaN,676357.0,NaN,180931.0,2300.0,5974.0,100.0,29716907.0,203623.0,Europe
4,South Africa,Africa,5.938157e+07,538184,NaN,9604.0,NaN,387316.0,NaN,141264.0,539.0,9063.0,162.0,3149807.0,53044.0,Africa


#### Definición de esquemas

In [15]:
schema_country_wise_latest = {
    "Country/Region": str,
    "Confirmed": int,
    "Deaths": int,
    "Recovered": int,
    "Active": int,
    "New cases": int,
    "New deaths": int,
    "New recovered": int,
    "Deaths / 100 Cases": float,
    "Recovered / 100 Cases": float,
    "Deaths / 100 Recovered": float,
    "Confirmed last week": int,
    "1 week change": int,
    "1 week % increase": float,
    "WHO Region": str
}

In [16]:
schema_covid_19_clean_complete = {
    "Province/State": str,
    "Country/Region": str,
    "Lat": float,
    "Long": float,
    "Date": str, 
    "Confirmed": int,
    "Deaths": int,
    "Recovered": int,
    "Active": int,
    "WHO Region": str
}

In [17]:
schema_day_wise = {
    "Date": str,  
    "Confirmed": int,
    "Deaths": int,
    "Recovered": int,
    "Active": int,
    "New cases": int,
    "New deaths": float,
    "New recovered": int,
    "Deaths / 100 Cases": float,
    "Recovered / 100 Cases": float,
    "Deaths / 100 Recovered": float,
    "No. of countries": int
}

In [18]:
schema_full_grouped = {
    "Date": str,  
    "Country/Region": str,
    "Confirmed": int,
    "Deaths": int,
    "Recovered": int,
    "Active": int,
    "New cases": int,
    "New deaths": int,
    "New recovered": int,
    "WHO Region": str
}

In [19]:
schema_usa_county_wise = {
    "UID": int,
    "iso2": str,
    "iso3": str,
    "code3": int,
    "FIPS": int,
    "Admin2": str,
    "Province_State": str,
    "Country_Region": str,
    "Lat": float,  
    "Long_": float,  
    "Combined_Key": str,
    "Date": str,  
    "Confirmed": int,
    "Deaths": int
}

In [20]:
schema_worldometer_data = {
    "Country/Region": str,
    "Continent": str,
    "Population": int,
    "TotalCases": int,
    "NewCases": int,
    "TotalDeaths": int,
    "NewDeaths": int,
    "TotalRecovered": int,
    "NewRecovered": int,
    "ActiveCases": int,
    "Serious,Critical": int,
    "Tot Cases/1M pop": int,
    "Deaths/1M pop": int,
    "TotalTests": int,
    "Tests/1M pop": int,
    "WHO Region": str
}

#### Transformación de datos

In [21]:
 list_schemas = [schema_country_wise_latest
                , schema_covid_19_clean_complete
                , schema_day_wise
                , schema_full_grouped
                , schema_usa_county_wise
                , schema_worldometer_data
            ]
    
list_dataframes = [country_wise_latest_df
                , covid_19_clean_complete_df
                , day_wise_df
                , full_grouped_df
                , usa_county_wise_df
                , worldometer_data_df
             ]

#### Tratamiento de datos

###### Como no es posible castear las columnas del dataframe de float a int habiendo valores nulos en la misma, debo tomar una decisión de qué hacer con estos valores.En este caso, casteo la columna a float, aunque habría opciones como reemplazar el valor nulo, o dropear los registros.

In [22]:
for schema, dataframe in zip(list_schemas, list_dataframes):
    for columna, tipo in schema.items():
        try:
            dataframe[columna] = dataframe[columna].astype(tipo)
        except ValueError:
            dataframe[columna] = dataframe[columna].astype(float)

##### Valido el resultado de la transformación

In [23]:
country_wise_latest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country/Region          187 non-null    object 
 1   Confirmed               187 non-null    int32  
 2   Deaths                  187 non-null    int32  
 3   Recovered               187 non-null    int32  
 4   Active                  187 non-null    int32  
 5   New cases               187 non-null    int32  
 6   New deaths              187 non-null    int32  
 7   New recovered           187 non-null    int32  
 8   Deaths / 100 Cases      187 non-null    float64
 9   Recovered / 100 Cases   187 non-null    float64
 10  Deaths / 100 Recovered  187 non-null    float64
 11  Confirmed last week     187 non-null    int32  
 12  1 week change           187 non-null    int32  
 13  1 week % increase       187 non-null    float64
 14  WHO Region              187 non-null    ob

In [24]:
covid_19_clean_complete_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49068 entries, 0 to 49067
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  49068 non-null  object 
 1   Country/Region  49068 non-null  object 
 2   Lat             49068 non-null  float64
 3   Long            49068 non-null  float64
 4   Date            49068 non-null  object 
 5   Confirmed       49068 non-null  int32  
 6   Deaths          49068 non-null  int32  
 7   Recovered       49068 non-null  int32  
 8   Active          49068 non-null  int32  
 9   WHO Region      49068 non-null  object 
dtypes: float64(2), int32(4), object(4)
memory usage: 3.0+ MB


In [25]:
day_wise_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    188 non-null    object 
 1   Confirmed               188 non-null    int32  
 2   Deaths                  188 non-null    int32  
 3   Recovered               188 non-null    int32  
 4   Active                  188 non-null    int32  
 5   New cases               188 non-null    int32  
 6   New deaths              188 non-null    float64
 7   New recovered           188 non-null    int32  
 8   Deaths / 100 Cases      188 non-null    float64
 9   Recovered / 100 Cases   188 non-null    float64
 10  Deaths / 100 Recovered  188 non-null    float64
 11  No. of countries        188 non-null    int32  
dtypes: float64(4), int32(7), object(1)
memory usage: 12.6+ KB


In [26]:
full_grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35156 entries, 0 to 35155
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            35156 non-null  object
 1   Country/Region  35156 non-null  object
 2   Confirmed       35156 non-null  int32 
 3   Deaths          35156 non-null  int32 
 4   Recovered       35156 non-null  int32 
 5   Active          35156 non-null  int32 
 6   New cases       35156 non-null  int32 
 7   New deaths      35156 non-null  int32 
 8   New recovered   35156 non-null  int32 
 9   WHO Region      35156 non-null  object
dtypes: int32(7), object(3)
memory usage: 1.7+ MB


In [27]:
usa_county_wise_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627920 entries, 0 to 627919
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   UID             627920 non-null  int32  
 1   iso2            627920 non-null  object 
 2   iso3            627920 non-null  object 
 3   code3           627920 non-null  int32  
 4   FIPS            626040 non-null  float64
 5   Admin2          627920 non-null  object 
 6   Province_State  627920 non-null  object 
 7   Country_Region  627920 non-null  object 
 8   Lat             627920 non-null  float64
 9   Long_           627920 non-null  float64
 10  Combined_Key    627920 non-null  object 
 11  Date            627920 non-null  object 
 12  Confirmed       627920 non-null  int32  
 13  Deaths          627920 non-null  int32  
dtypes: float64(3), int32(4), object(7)
memory usage: 57.5+ MB


In [28]:
worldometer_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country/Region    209 non-null    object 
 1   Continent         209 non-null    object 
 2   Population        208 non-null    float64
 3   TotalCases        209 non-null    int32  
 4   NewCases          4 non-null      float64
 5   TotalDeaths       188 non-null    float64
 6   NewDeaths         3 non-null      float64
 7   TotalRecovered    205 non-null    float64
 8   NewRecovered      3 non-null      float64
 9   ActiveCases       205 non-null    float64
 10  Serious,Critical  122 non-null    float64
 11  Tot Cases/1M pop  208 non-null    float64
 12  Deaths/1M pop     187 non-null    float64
 13  TotalTests        191 non-null    float64
 14  Tests/1M pop      191 non-null    float64
 15  WHO Region        209 non-null    object 
dtypes: float64(12), int32(1), object(3)
memory u

###### Se puede observar como al castear los datos al tipo correcto, el uso de memoria se reduce.

#### Persistencia de datos

In [29]:
path_output = "output/kaggle/pandas/"

In [30]:
country_wise_latest_df.to_parquet(f"{path_output}/{file_name_country_wise_latest}/{file_name_country_wise_latest}.parquet", index = False)

covid_19_clean_complete_df.to_parquet(f"{path_output}/{file_name_covid_19_clean_complete_df}/{file_name_covid_19_clean_complete_df}.parquet", index = False)

day_wise_df.to_parquet(f"{path_output}/{file_name_day_wise_df}/{file_name_day_wise_df}.parquet", index = False)

full_grouped_df.to_parquet(f"{path_output}/{file_name_full_grouped_df}/{file_name_full_grouped_df}.parquet", index = False)

usa_county_wise_df.to_parquet(f"{path_output}/{file_name_usa_county_wise_df}/{file_name_usa_county_wise_df}.parquet", index = False)

worldometer_data_df.to_parquet(f"{path_output}/{file_name_worldometer_data_df}/{file_name_worldometer_data_df}.parquet", index = False)